In [1]:
import pandas as pd

In [2]:
user = pd.read_csv('user_7.csv')

In [3]:
user_session = pd.read_csv('user_session_7.csv')

## Предобработка данных

In [4]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1000 non-null   int64 
 1   gender        1000 non-null   object
 2   birthday      1000 non-null   object
 3   region        1000 non-null   object
 4   last_visited  1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [5]:
user.describe(include='object')

,gender,birthday,region,last_visited
count,1000,1000,1000,1000
unique,2,1000,5,1000
top,Ж,2007-07-15 18:53:02,Город Москва,2023-09-20 18:00:34.464576
freq,575,1,451,1


In [6]:
user.gender = user.gender.astype('category')
user.region	= user.region.astype('category')

In [7]:
user.birthday = pd.to_datetime(user.birthday)
user.last_visited = pd.to_datetime(user.last_visited)

In [8]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1000 non-null   int64         
 1   gender        1000 non-null   category      
 2   birthday      1000 non-null   datetime64[ns]
 3   region        1000 non-null   category      
 4   last_visited  1000 non-null   datetime64[ns]
dtypes: category(2), datetime64[ns](2), int64(1)
memory usage: 25.8 KB


In [9]:
user_session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52612 entries, 0 to 52611
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   session_id    52612 non-null  int64 
 1   user_id       52612 non-null  int64 
 2   date          52612 non-null  object
 3   time_watched  52612 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.6+ MB


In [10]:
user_session.date = pd.to_datetime(user_session.date)

In [11]:
user_session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52612 entries, 0 to 52611
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   session_id    52612 non-null  int64         
 1   user_id       52612 non-null  int64         
 2   date          52612 non-null  datetime64[ns]
 3   time_watched  52612 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.6 MB


In [12]:
df = pd.merge(
        user,
        user_session,
        left_on='id',
        right_on='user_id',
        validate='1:m'
)

In [13]:
df = df.drop('user_id', axis=1)
df = df.rename(columns={'id': 'user_id'})

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52612 entries, 0 to 52611
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       52612 non-null  int64         
 1   gender        52612 non-null  category      
 2   birthday      52612 non-null  datetime64[ns]
 3   region        52612 non-null  category      
 4   last_visited  52612 non-null  datetime64[ns]
 5   session_id    52612 non-null  int64         
 6   date          52612 non-null  datetime64[ns]
 7   time_watched  52612 non-null  int64         
dtypes: category(2), datetime64[ns](3), int64(3)
memory usage: 2.9 MB


## Задача 1

Посчитайте долю пользователей мужского пола в таблице `users`.

Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для подсчёта доли объектов, удовлетворяющих заданному условию, удобно использовать функцию `mean`, вызвав её от колонки-фильтра со значениями булевого типа. В данном случае каждое знаечние `True` будет приведено к $1$, а каждое значение `False` — к $0$. Усреднение этих значений как раз и даст долю объектов, удволетворяющих заданному условию.

In [9]:
(user.gender == 'М').mean().round(2)

0.42

In [12]:
user.gender.value_counts()

Ж    575
М    425
Name: gender, dtype: int64

In [13]:
## чертово питоновское банковское округление :)))

## Задача 2

Посчитайте долю пользователей из Краснодарского края, которые заходили на платформу в сентябре $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [32]:
round(user[
            (user.region == 'Краснодарский край') 
            & (user.last_visited.dt.to_period('m') == '2023-09')
      ].id.nunique() 
      / user[user.region == 'Краснодарский край'].id.nunique()
     , 2)

0.41

## Задача 3

По таблице `user_session` рассчитайте долю активных пользователей в мае $2023$-го года относительно общего числа пользователей платформы из таблицы `users`.

Активными считайте пользователей, у которых в этот месяц была хотя бы одна сессия.

Ответ округлите до $2$-х знаков после запятой.

In [36]:
round(
        user_session[user_session.date.dt.to_period('m') == '2023-05'].user_id.nunique() 
                / user.id.nunique()
        , 2
)

0.6

## Задача 4

Дадим другое определение активным пользователям. Будем считать активными в конкретном месяце тех, кто посмотрел хотя бы $60$ минут видеоконтента в этом месяце.

Рассчитайте долю активных пользователей среди тех, у кого была хотя бы одна сессия в марте $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [45]:
user_time = user_session[user_session.date.dt.to_period('m') == '2023-03'].groupby('user_id').time_watched.sum()

In [49]:
(user_time > 60).mean().round(2)

0.92

# Задачи 5-9

Рассчитайте, сколько в среднем смотрят видеоконтента пользователи за один день активности. Днём активности называем день, в который у пользователя была сессия.

## Задача 5

Для этого сначала вычислите суммарное число дней, когда пользователи смотрели видеоконтент.

Обратите внимание, что хараткеристику можно посчитать двумя способами. Рассчитатйте её как число уникальных пар вида (пользователь; день, когда у него была сессия).

In [15]:
user.head(1)

,id,gender,birthday,region,last_visited
0,1,М,2007-07-15 18:53:02,Город Москва,2023-09-20 18:00:34.464576


In [16]:
user_session.head(1)

,session_id,user_id,date,time_watched
0,1,558,2023-01-23 00:52:14,10


In [17]:
df.head(1)

,user_id,gender,birthday,region,last_visited,session_id,date,time_watched
0,1,М,2007-07-15 18:53:02,Город Москва,2023-09-20 18:00:34.464576,4021,2023-02-05 16:28:36,31


## Задача 6

Теперь вычислите суммарное время просмотра видеоконтента пользователями.

## Задача 7

Вычислите среднее количество просмотренного пользователями видеоконтента за день активности, как отношение суммарного времени просмотра к суммарному числу дней просмотра.

Ответ округлите до $2$-х знаков после запятой.

## Задача 8

Обратите внимание, что есть альтернативный способ рассчитать, сколько в среднем видеоконтента смотрят пользователи за один день активности.

Во-первых, рассчитайте для каждого из пользователей среднее количество просмотренного им видеоконтента. Принцип вычисления метрики для конкретного пользователя такой же, как был для всех пользователей вместе: нужно взять суммарное время просмотра контента и разделить на суммарное количество уникальных дней активности.

В качестве ответа укажите среднее количество просмотренного видеоконтента пользователем с `id`, равным $738$. Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для того чтобы вычислить среднее количество видеоконтента, просмотренного конкретным пользователем, удобно провести группировку по пользователям и произвести агрегацию по двум колонкам отдельно: по колонке `time_watched`, просуммировав значения в ней , и по колонке `date`, посчитав количество уникальных дней активности для каждого из пользователей.

Для того чтобы произвести агрегацию по колонке `date`, удобно воспользоваться лямбда-функцией, которая сначала произведёт округление моментов времени из колонке `date` _вниз_ до дней, а потом посчитает количество полученных уникальных дней.

## Задача 9

Для того чтобы вычислить среднее время просмотра видеоконтента за день активности, посчитайте среднее арифметическое данной характеристики по всем пользователям. Для этого используйте данные, полученные в предыдущем упражнении.

Ответ округлите до $2$-х знаков после запятой.